In [1]:
import re
import matplotlib.pyplot as plt
from io import BytesIO
import openpyxl
import json
from datetime import datetime
from telegram.ext import Updater, MessageHandler, Filters
import os
from dotenv import load_dotenv


TOKEN = os.getenv("TELEGRAM_TOKEN")

with open("categorias.json", "r", encoding="utf-8") as f:
    categorias_dict = json.load(f)

try:
    workbook = openpyxl.load_workbook("financas.xlsx")
    sheet = workbook.active
except FileNotFoundError:
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(["Data", "Descrição", "Valor", "Tipo", "Categoria"])

def lista_categorias():
    texto = ""
    for cat, palavras in categorias_dict.items():
        texto += f"- {cat}: {', '.join(palavras)}\n"
    return texto

import unicodedata

def normalizar(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII').lower()

def classificar_mensagem(mensagem):
    mensagem_norm = normalizar(mensagem)
    valor = None
    tipo = None
    categoria = None
    subcategoria = None
    encontrado = re.search(r"(\d+(?:,\d{2})?)", mensagem_norm)
    if encontrado:
        valor = float(encontrado.group(1).replace(",", "."))
    # Gastos
    for cat, subs in categorias_dict.get("Gastos", {}).items():
        for subcat, palavras in subs.items():
            for palavra in palavras:
                palavra_norm = normalizar(palavra)
                if re.search(rf"\b{re.escape(palavra_norm)}\b", mensagem_norm):
                    categoria = cat
                    subcategoria = subcat
                    tipo = "Gasto"
                    break
            if categoria:
                break
        if categoria:
            break
    # Ganhos
    if not categoria:
        for subcat, palavras in categorias_dict.get("Ganhos", {}).items():
            for palavra in palavras:
                palavra_norm = normalizar(palavra)
                if re.search(rf"\b{re.escape(palavra_norm)}\b", mensagem_norm):
                    categoria = "Ganhos"
                    subcategoria = subcat
                    tipo = "Ganho"
                    break
            if categoria == "Ganhos":
                break
    # Retorna subcategoria junto
    return valor, tipo, categoria, subcategoria

def processar_linhas(texto):
    respostas = []
    linhas = texto.splitlines()
    for linha in linhas:
        if not linha.strip():  # Pula a linha se for vazia
            continue
        valor, tipo, categoria, subcategoria = classificar_mensagem(linha)
        if valor and tipo and categoria and subcategoria:
            sheet.append([datetime.now().strftime("%d/%m/%Y"), linha, valor, tipo, f"{categoria} > {subcategoria}"])
            respostas.append(f"✅ Registrado: {valor} - {tipo} - {categoria} > {subcategoria}")
        elif valor:
            respostas.append(
                f"❌ Categoria não reconhecida para '{linha}'. Veja as opções:\n{lista_categorias()}\nOu envie uma nova categoria."
            )
        else:
            respostas.append(f"❌ Não entendi o valor em: '{linha}'")
    workbook.save("financas.xlsx")
    return "\n".join(respostas)


def boas_vindas(update, context):
    texto = (
        "👋 Olá! Bem-vindo ao seu bot de finanças!\n\n"
        "Você pode:\n"
        "- Visualizar registros: escreva 'visualizar'\n"
        "- Ver relatório: escreva 'relatorio'\n"
        "- Alterar categoria: escreva 'alterar <número> <nova_categoria>'\n"
        "- Remover registros: escreva 'remover <número(s)>'\n"
        "Para registrar gastos/ganhos, envie a descrição normalmente (pode ser várias linhas).\n"
        "Se quiser ver as categorias disponíveis, escreva 'categorias'."
    )
    update.message.reply_text(texto)

def receber_mensagem(update, context):
    msg = update.message.text.strip()
    if msg.lower() == "visualizar":
        visualizar(update, context)
    elif msg.lower().startswith("relatorio"):
        relatorio(update, context)
    elif msg.lower().startswith("alterar"):
        alterar(update, context)
    elif msg.lower().startswith("remover"):
        remover(update, context)
    elif msg.lower() == "categorias":
        update.message.reply_text(f"Categorias disponíveis:\n{lista_categorias()}")
    elif msg.lower() in ["oi", "olá", "inicio", "start"]:
        boas_vindas(update, context)
    else:
        resposta = processar_linhas(msg)
        update.message.reply_text(resposta)


def relatorio(update, context):
    args = update.message.text.strip().split()
    tipo_grafico = args[1].lower() if len(args) > 1 else None

    gastos = {}
    ganhos = 0
    total_gastos = 0

    for row in sheet.iter_rows(min_row=2, values_only=True):
        data, desc, valor, tipo, categoria = row
        if tipo == "Gasto":
            gastos[categoria] = gastos.get(categoria, 0) + valor
            total_gastos += valor
        elif tipo == "Ganho":
            ganhos += valor

    saldo = ganhos - total_gastos

    # Texto do relatório
    texto = "📊 *Relatório Financeiro*\n\n"
    texto += f"💰 *Ganhos:* R$ {ganhos:.2f}\n"
    texto += f"💸 *Gastos:* R$ {total_gastos:.2f}\n"
    texto += f"⚖️ *Saldo:* {'🟢' if saldo >= 0 else '🔴'} R$ {saldo:.2f}\n\n"

    if gastos:
        texto += "*📂 Gastos por categoria:*\n"
        for cat, total in sorted(gastos.items(), key=lambda x: x[1], reverse=True):
            porcent = (total / total_gastos * 100) if total_gastos > 0 else 0
            barra = "█" * int(porcent // 5)
            texto += f"- {cat}: R$ {total:.2f} ({porcent:.1f}%) {barra}\n"

        # Sempre manda texto
        update.message.reply_text(texto, parse_mode="Markdown")

        # Só gera gráfico se pedido
        if tipo_grafico in ["pizza", "barra"]:
            fig, ax = plt.subplots(figsize=(7, 5))
            categorias = list(gastos.keys())
            valores = list(gastos.values())

            if tipo_grafico == "barra":
                ax.bar(categorias, valores)
                ax.set_title("Gastos por Categoria", fontsize=14)
                ax.set_ylabel("Valor (R$)")
                ax.set_xlabel("Categorias")
                plt.xticks(rotation=30, ha="right")
            elif tipo_grafico == "pizza":
                ax.pie(valores, labels=categorias, autopct='%1.1f%%', startangle=140)
                ax.set_title("Distribuição de Gastos", fontsize=14)

            buf = BytesIO()
            plt.savefig(buf, format="png", bbox_inches="tight")
            buf.seek(0)
            plt.close(fig)

            update.message.reply_photo(photo=buf)

    else:
        texto += "Nenhum gasto registrado ainda ✅"
        update.message.reply_text(texto, parse_mode="Markdown")



def visualizar(update, context):
    texto = "📋 *Registros:*\n"
    for i, row in enumerate(sheet.iter_rows(min_row=2, values_only=True), start=1):
        data, desc, valor, tipo, categoria = row
        texto += f"{i}. {data} | {desc} | R$ {valor:.2f} | {tipo} | {categoria}\n"
    update.message.reply_text(texto, parse_mode="Markdown")

def alterar(update, context):
    try:
        args = update.message.text.strip().split()
        idx = int(args[1])
        nova_categoria = args[2]
        row = list(sheet.iter_rows(min_row=2, max_row=idx+1, values_only=False))[-1]
        row[4].value = nova_categoria
        workbook.save("financas.xlsx")
        update.message.reply_text(f"✅ Categoria alterada para {nova_categoria} no registro {idx}.")
    except Exception:
        update.message.reply_text("❌ Use: alterar <número> <nova_categoria>")

def remover(update, context):
    try:
        args = update.message.text.strip().split()
        indices = [int(i) for i in args[1:]]
        for idx in sorted(indices, reverse=True):
            sheet.delete_rows(idx+1)
        workbook.save("financas.xlsx")
        update.message.reply_text(f"✅ Registros removidos: {', '.join(map(str, indices))}")
    except Exception:
        update.message.reply_text("❌ Use: remover <número(s)> (ex: remover 1 2 3)")

updater = Updater(TOKEN, use_context=True)
dispatcher = updater.dispatcher
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, receber_mensagem))

print("🤖 Bot rodando...")
updater.start_polling()
updater.idle()

c:\Users\alice\OneDrive\Área de Trabalho\Projetos\Chatbot de finanças\.venv312\Lib\site-packages\telegram\utils\request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(
c:\Users\alice\OneDrive\Área de Trabalho\Projetos\Chatbot de finanças\.venv312\Lib\site-packages\apscheduler\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


🤖 Bot rodando...
